## Predictive model - Classification

In this notebook, optimisation of marketing campaign is made by training machine learning model and analysis of features estimated to be the most responsible for the final predictions

#### Data Preprocessing
1. Filling missing values

2. Resolving correlation between euribor3m and emp.var.rate variables??? - Try later

In the original work of S. Moro et al, both variables were used. Moreover, both of them appeared to be among top8 most important features (euribor3m on the 1st place, and emp.var.rate on the 8th)

3. Transformation of numerical variables into categorical ones??? - Try later
4. Encoding categorical varibles/transforming into dumnies columns
5. Resampling for train/test splits due to unbalanced targets

#### ML training and testing
In the original work published in 2014, the best results were obtained with NN ensemble composed of Nr = 7 distinct networks, each trained with 100 epochs of the BFGS algorithm. The number of hidden nodes was H = round(M/2) (M is the number of inputs). Among other tested models there were LogisticRegression, SVM and RandomForest.

From the view point of results interpretibility, most promissing current alternatives are boosted models, particularly XGBoost, CatBoost and LightGBM classifiers

http://archive.ics.uci.edu/ml/datasets/Bank+Marketing
    
**Data Set Information:**

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be ('yes') or not ('no') subscribed.

There are four datasets:

 1. bank-additional-full.csv with all examples (41188) and 20 inputs, ordered by date (from May 2008 to November 2010), very close to the data analyzed in [Moro et al., 2014]
 2. bank-additional.csv with 10% of the examples (4119), randomly selected from 1), and 20 inputs.
 3. bank-full.csv with all examples and 17 inputs, ordered by date (older version of this dataset with less inputs).
 4. bank.csv with 10% of the examples and 17 inputs, randomly selected from 3 (older version of this dataset with less inputs).
 
The smallest datasets are provided to test more computationally demanding machine learning algorithms (e.g., SVM).

The classification goal is to predict if the client will subscribe (yes/no) a term deposit (variable y).



### Attribute Information:

Input variables:
#### bank client data:

1.  **age** (numeric)
2.  **job** : type of job (categorical: 'admin.','bluecollar','entrepreneur','housemaid','management','retired','selfemployed','services','student','technician','unemployed','unknown')
3.  **marital** : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4.  **education** (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5.  **default**: has credit in default? (categorical: 'no','yes','unknown')
6.  **housing**: has housing loan? (categorical: 'no','yes','unknown')
7.  **loan**: has personal loan? (categorical: 'no','yes','unknown')

#### related with the last contact of the current campaign:
8.  **contact**: contact communication type (categorical: 'cellular','telephone')
9.  **month**: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10.  **day_of_week**: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11.  **duration**: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

#### other attributes:
12.  **campaign**: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13.  **pdays**: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14.  **previous**: number of contacts performed before this campaign and for this client (numeric)
15.  **poutcome**: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

#### social and economic context attributes
16.  **emp.var.rate**: employment variation rate  quarterly indicator (numeric)
17.  **cons.price.idx**: consumer price index  monthly indicator (numeric)
18.  **cons.conf.idx**: consumer confidence index  monthly indicator (numeric)
19.  **euribor3m**: euribor 3 month rate  daily indicator (numeric)
20.  **nr.employed**: number of employees  quarterly indicator (numeric)

Output variable (desired target):
21.  **y**  has the client subscribed a term deposit? (binary: 'yes','no')

In [3]:
# import all the libraries

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd 

import random
random.seed(123)

from IPython.display import display
import random
import time

pd.options.display.max_columns = None
import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [28]:
df = pd.read_csv("bank-additional/bank-additional-full.csv", sep =";")
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


## Data Preprocessing

In [29]:
map_dict= {"no":0, "yes":1}
df["y"] = df["y"].map(map_dict)

### Filling missing values

In [30]:
text_cat_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact']

In [40]:
df.groupby(["y","default"])[["age"]].count()

age
y default       
0 no       28391
  unknown   8154
  yes          3
1 no        4197
  unknown    443

The most reasonable way here would be fill with "no" values, but in that case we need to drop the entire column, as it will have only "no" values. 

So it will be filled randomly with "yes" and "no" values

In [37]:
cat_values_dict = {}
for col in text_cat_columns:
    print("Column: %s, Number of unique values: %d"% (col, df[col].nunique()))
    cat_values_dict[col] = list(set(df[col].unique())-set(["unknown"]))

cat_values_dict

Column: job, Number of unique values: 11
Column: marital, Number of unique values: 3
Column: education, Number of unique values: 7
Column: default, Number of unique values: 3
Column: housing, Number of unique values: 2
Column: loan, Number of unique values: 2
Column: contact, Number of unique values: 2


{'job': ['technician',
  'self-employed',
  'student',
  'housemaid',
  'entrepreneur',
  'retired',
  'management',
  'unemployed',
  'admin.',
  'blue-collar',
  'services'],
 'marital': ['married', 'divorced', 'single'],
 'education': ['professional.course',
  'basic.9y',
  'university.degree',
  'basic.4y',
  'illiterate',
  'high.school',
  'basic.6y'],
 'default': ['yes', 'no'],
 'housing': ['no', 'yes'],
 'loan': ['no', 'yes'],
 'contact': ['cellular', 'telephone']}

In [41]:
# filling with random values from lists of availbale categories

for col in text_cat_columns:
    n_missing = df[df[col]=="unknown"].shape[0]
    missing_index = df[df[col]=="unknown"].index
    print(col, n_missing)
    fill_vallues = cat_values_dict[col]*n_missing
    random.shuffle(fill_vallues)
    df.loc[missing_index, col]= fill_vallues[:n_missing]

job 0
marital 0
education 0
default 8597
housing 0
loan 0
contact 0


In [42]:
df.groupby(["y","default"])[["age"]].count()

age
y default       
0 no       32479
  yes       4069
1 no        4434
  yes        206

### Encoding categories & Correlated predictors

Possible scenarios:
1. leave correlated predictors (as it was done in the original work) and create dummies columns for all categorical variables
2. leave correlated predictors (as it was done in the original work) and make OHE/Label encoding
3. drop one of the correlated features and make OHE/Label encoding

Considering our final task on giving recommendations on optimisation, the first scenario seem to provide more valuable information. Therefore first approach is tested in this notebook

In [47]:
# OHE in one column
df2 = df.copy()
map_yes_no_dict = {"yes": 1, "no":0}
map_phone_dict = {"cellular": 1, "telephone":0}
for col in ['default', 'housing','loan']:
    df2[col] = df2[col].map(map_yes_no_dict)
df2['contact'] = df2['contact'].map(map_phone_dict) 


In [111]:
# for all the rest - dummies columns
df1 = pd.get_dummies(df)
print("Original data: {}, Encoded data: {}".format(df.shape, df2.shape))

Original data: (41188, 21), Encoded data: (41188, 54)


Number of features increased by 33 columns

## Train/Test split

https://www.kaggle.com/rafjaa/resampling-strategies-for-imbalanced-datasets

For separating a train set  it is more reasonable to use resampling
which means removing samples from the majority class (under-sampling) and 
adding more examples from the minority class (over-sampling).

In [112]:
y_0, y_1 = df["y"].value_counts()
y_0, y_1, round(y_0/y_1, 2)

(36548, 4640, 7.88)

In [113]:
df_0 = df[df["y"] == 0]
df_1 = df[df["y"] == 1]

In [114]:
from sklearn.model_selection import train_test_split

In [115]:
X = df1.drop("y", axis=1)
y = df1["y"]

In [117]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify= y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, stratify= y_train)
X_train.shape, y_train.shape

In [122]:
print("Original proportion of classes:", round(df1[df1["y"]==0].shape[0]/df1[df1["y"]==1].shape[0], 2)) 
print("Proportion in test data after splitting:", round(len(y_test)/np.sum(y_test[y_test ==1]), 2))

Original proportion of classes: 7.88
Proportion in test data after splitting: 8.88


## Approach 1. Resampling

In [105]:
# SMOTE (Synthetic Minority Oversampling TEchnique) consists of synthesizing elements for the minority class, 
# based on those that already exist. It works randomly picingk a point from the minority class 
# and computing the k-nearest neighbors for this point. 
# The synthetic points are added between the chosen point and its neighbors

from imblearn.combine import SMOTETomek

smotemek = SMOTETomek(sampling_strategy='auto')

# fit the object to our training data.
X_train2, y_train2 = smotemek.fit_sample(X_train, y_train)

len(y_train2[y_train2==0]), len(y_train2[y_train2==1]), len(y_train)

(28997, 28997, 32950)

In [107]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import time
from sklearn.preprocessing import StandardScaler

In [123]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.fit_transform(X_val)
X_test = scaler.transform(X_test)

In [125]:
def print_models_results(my_cv_results):
    print("Best Parameters: {}\n".format(my_cv_results.best_params_))
    means = my_cv_results.cv_results_["mean_test_score"]
    stds = my_cv_results.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, my_cv_results.cv_results_["params"]):
        print("{} +/- {} for {}".format(round(mean, 3), round(std*2, 3), params))

In [ ]:
%%time
rf = RandomForestClassifier()
parameters = {"n_estimators" : [5, 50, 250],
             "max_depth": [2,4,8,16, None]}

rf_cv = GridSearchCV(rf, parameters, cv=5) #by defult refit =True, so the best model is automatically retrained on the whole data
rf_cv.fit(X, y.ravel())

print_models_results(rf_cv)
rf_best = rf_cv.best_estimator_

In [ ]:
%%time
gb = GradientBoostingClassifier()
parameters = {"n_estimators" : [5, 50, 250],
             "max_depth": [1, 3, 5, 7],
             "learning_rate": [0.01, 0.1, 1, 10]}

gb_cv = GridSearchCV(gb, parameters, cv=splits)
gb_cv.fit(X, y.ravel())

print_models_results(gb_cv)
gb_best = gb_cv.best_estimator_

In [126]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
model_names = ["RF", "GB"]
results_df = pd.DataFrame(index= model_names, columns = ["precision", "recall", "f1_score"])
models = [rf_best, gb_best]

for i, model in enumerate(models):
    y_pred = models[i].predict(X_val)
    results_df.loc[model_names[i], "precision"] = precision_score(y_test, y_pred)
    results_df.loc[model_names[i], "recall"] = recall_score(y_test, y_pred)
    results_df.loc[model_names[i], "f1_score"] = f1_score(y_test, y_pred)
results_df

In [ ]:
feature_importances = pd.DataFrame(rf_best.feature_importances_, 
                                   index = df2.drop('campaign_7',axis=1).columns, 
                                   columns=['importance_RF'])

feature_importances["importance_GB"] = gb_best.feature_importances_
feature_importances[feature_importances["importance_GB"]!=0]